In [1]:
list_features = []
for i in range (0,4):
    for j in range (0,4):
        for k in range (0,4):
            for w in range (0,4):
                list_features.append([i,j,k,w])
#print (np.shape(list_features))

In [2]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############
def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        # join all sequence lines to one.
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data1=[]
training_data2=[]


######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        temp.append(temp1)
    training_data1.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        #print (temp1)
        temp.append(temp1)
    training_data2.append(temp)
#training_data
list_freq1 = []
list_freq2 = []

for i in training_data1:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq1.append(temp)
    
for i in training_data2:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq2.append(temp)    

In [3]:
# print(np.shape(list_freq1))
# a-0 t-1 g-2 c-3
ans=[]
cnt=0
for i in range(256):
    flag=0
    temp=list_features[i]
    for j in range(838):
        if (list_freq1[j][i]==0):
            flag=1
            break
    if(flag==1):
        cnt+=1
    if (flag==0):
        ans.append(temp)
    
print(ans)
print(cnt)


[]
256


In [26]:
ans=np.sum(list_freq1,axis=0)
ans=np.divide(ans,8.39)
out=[]
for i in range(256):
    if(ans[i]>70):
        out.append([list_features[i],ans[i]])

out

[[[0, 0, 0, 0], 102.97973778307508],
 [[0, 0, 0, 1], 79.26102502979737],
 [[0, 0, 1, 1], 73.06317044100119],
 [[0, 1, 0, 0], 73.7783075089392],
 [[0, 1, 1, 1], 85.10131108462454],
 [[1, 0, 0, 0], 72.82479141835518],
 [[1, 1, 0, 1], 75.92371871275327],
 [[1, 1, 1, 0], 76.51966626936829],
 [[1, 1, 1, 1], 114.18355184743741]]

In [27]:
ans=np.sum(list_freq2,axis=0)
ans=np.divide(ans,30)
out=[]
for i in range(256):
    if(ans[i]>70):
        out.append([list_features[i],ans[i]])
print(np.shape(list_freq2))
out

(3000, 256)


[[[2, 3, 3, 0], 71.8],
 [[3, 0, 2, 3], 88.3],
 [[3, 2, 3, 3], 80.3],
 [[3, 3, 0, 2], 87.06666666666666]]

In [13]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels1=[]
labels2=[]

for i in range(839):
    labels1.append(1)
for i in range(3000):
    labels2.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(list_freq1,labels1))
random.shuffle(comb)
list_freq1,labels1=zip(*comb)

comb=list(zip(list_freq2,labels2))
random.shuffle(comb)
list_freq2,labels2=zip(*comb)

print(np.shape(list_freq1))
print(np.shape(labels1))

#training_data=np.reshape(training_data,(3839,77*16))

#print(np.shape(training_data))


(839, 256)
(839,)


In [14]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=list()
Y_train=list()
for i in range(649):
    X_train.append(list_freq1[i])
    Y_train.append(labels1[i])
for i in range(2480):
    X_train.append(list_freq2[i])
    Y_train.append(labels2[i])


# X_train.append(list(list_freq1[:649]))
# Y_train.append(list(labels1[:649]))
# X_train.append(list(list_freq2[:2480]))
# Y_train.append(list(labels2[:2480]))

comb=list(zip(X_train,Y_train))
random.shuffle(comb)
X_train,Y_train=zip(*comb)

X_validation=[]
Y_validation=[]

for i in range(649,720):
    X_validation.append(list_freq1[i])
    Y_validation.append(labels1[i])
for i in range(2480,2740):
    X_validation.append(list_freq2[i])
    Y_validation.append(labels2[i])
# X_validation.append(list_freq1[649:720])
# Y_validation.append(labels1[:720])
# X_validation.append(list_freq2[2480:2740])
# Y_validation.append(labels2[2480:2740])

comb=list(zip(X_validation,Y_validation))
random.shuffle(comb)
X_validation,Y_validation=zip(*comb)

X_test=[]
Y_test=[]
for i in range(720,839):
    X_test.append(list_freq1[i])
    Y_test.append(labels1[i])
for i in range(2740,3000):
    X_test.append(list_freq2[i])
    Y_test.append(labels2[i])
# X_test.append(list_freq1[720:])
# Y_test.append(labels1[720:])
# X_test.append(list_freq2[2740:])
# Y_test.append(labels2[2740:])

comb=list(zip(X_test,Y_test))
random.shuffle(comb)
X_test,Y_test=zip(*comb)

X_test=list(X_test)
X_validation=list(X_validation)
X_train=list(X_train)

Y_test=list(Y_test)
Y_validation=list(Y_validation)
Y_train=list(Y_train)


X_train = np.array(X_train)
X_valid=np.array(X_validation)
X_test=np.array(X_test)

Y_test=np.array(Y_test)
Y_valid=np.array(Y_validation)
Y_train=np.array(Y_train)

# X_train=list(X_train)
# np.shape(X_train)

In [15]:
#XG BOOST
#XG BOOST

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

parameters={
    "booster":'gbtree',
    'learning_rate':0.03,
    "n_estimators":1200,
    "max_depth":2,
    "seed":3,
    "min_child_weight":6,
    
    "colsample_bylevel":0.9,
    "colsample_bytree":0.9,
}

model = XGBClassifier(**parameters)
model.fit(X_train, Y_train)
model

# make predictions for test data



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=2, min_child_weight=6, missing=None, n_estimators=1200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=3, silent=True,
       subsample=1)

In [23]:

# for layer in model.layers:
#     weights=layer.get_weights()
#     print(weights)

xg_boost_weights=model.feature_importances_
xg_boost_weights=np.multiply(xg_boost_weights,100)
out_xg=[]
for i in range(256):
    if(xg_boost_weights[i]>2):
        out_xg.append([list_features[i],xg_boost_weights[i]])
# print(np.shape(list_freq2))
out_xg

[[[1, 0, 1, 0], 2.3723438],
 [[1, 0, 1, 1], 4.1347585],
 [[1, 1, 0, 1], 5.6835904],
 [[1, 1, 1, 0], 2.3079212],
 [[3, 0, 2, 3], 3.0149353],
 [[3, 1, 0, 1], 4.319747],
 [[3, 3, 0, 2], 3.33228]]

In [9]:
##RANDOM FOREST
##RANDOM FOREST

########### CREATING LABELS FOR THE OUTPUT ###########
labels1=[]
labels2=[]

for i in range(839):
    labels1.append(1)
for i in range(3000):
    labels2.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(list_freq1,labels1))
random.shuffle(comb)
list_freq1,labels1=zip(*comb)

comb=list(zip(list_freq2,labels2))
random.shuffle(comb)
list_freq2,labels2=zip(*comb)

print(np.shape(list_freq1))
print(np.shape(labels1))

#training_data=np.reshape(training_data,(3839,77*16))

#print(np.shape(training_data))


####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=list()
Y_train=list()
for i in range(649):
    X_train.append(list_freq1[i])
    Y_train.append(labels1[i])
for i in range(2480):
    X_train.append(list_freq2[i])
    Y_train.append(labels2[i])


# X_train.append(list(list_freq1[:649]))
# Y_train.append(list(labels1[:649]))
# X_train.append(list(list_freq2[:2480]))
# Y_train.append(list(labels2[:2480]))

comb=list(zip(X_train,Y_train))
random.shuffle(comb)
X_train,Y_train=zip(*comb)

X_validation=[]
Y_validation=[]

for i in range(649,720):
    X_validation.append(list_freq1[i])
    Y_validation.append(labels1[i])
for i in range(2480,2740):
    X_validation.append(list_freq2[i])
    Y_validation.append(labels2[i])
# X_validation.append(list_freq1[649:720])
# Y_validation.append(labels1[:720])
# X_validation.append(list_freq2[2480:2740])
# Y_validation.append(labels2[2480:2740])

comb=list(zip(X_validation,Y_validation))
random.shuffle(comb)
X_validation,Y_validation=zip(*comb)

X_test=[]
Y_test=[]
for i in range(720,839):
    X_test.append(list_freq1[i])
    Y_test.append(labels1[i])
for i in range(2740,3000):
    X_test.append(list_freq2[i])
    Y_test.append(labels2[i])
# X_test.append(list_freq1[720:])
# Y_test.append(labels1[720:])
# X_test.append(list_freq2[2740:])
# Y_test.append(labels2[2740:])

comb=list(zip(X_test,Y_test))
random.shuffle(comb)
X_test,Y_test=zip(*comb)


# X_train=list(X_train)
# np.shape(X_train)


from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV


parameters = {'bootstrap': True,
              'min_samples_leaf':6,
              'n_estimators': 500, 
              'min_samples_split': 10,
              'max_features': None,
              'max_depth': 20,
              'warm_start':True
             }

RF_model = RandomForestClassifier(**parameters)
RF_model.fit(X_train, Y_train)


(839, 256)
(839,)


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [33]:
rf_weights=RF_model.feature_importances_
rf_weights=np.multiply(rf_weights,100)
out_rf=[]
for i in range(256):
    if(rf_weights[i]>2):
        out_rf.append([list_features[i],rf_weights[i]])
# print(np.shape(list_freq2))
out_rf

[[[0, 0, 0, 0], 6.664584941559003],
 [[0, 0, 1, 1], 2.5095249643076345],
 [[1, 0, 1, 0], 2.160238287803165],
 [[1, 1, 0, 1], 15.751748560349052],
 [[3, 0, 2, 3], 5.3299538347423985],
 [[3, 1, 0, 1], 6.703437386078198],
 [[3, 2, 3, 3], 2.228481015260176],
 [[3, 3, 0, 2], 4.090312718606231]]